In [0]:
!pip install -U -q PyDrive
# !pip install -U pandas

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html

# 2. Create & upload a file text file.
# uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
# uploaded.SetContentString('Sample upload file content')
# uploaded.Upload()
# print('Uploaded file with ID {}'.format(uploaded.get('id')))


file_id = '1qivHbr7gndjKEoAWoAGltAVbZSe_6GaC'

# 3. Load a file by ID and print its contents.
# downloaded = drive.CreateFile({'id': uploaded.get('id')})
downloaded = drive.CreateFile({'id': file_id})
db = downloaded.GetContentString()


data = [d.replace("\r","").split(";") for d in db.split("\n") if d != ""]
print(data)

    100% |████████████████████████████████| 993kB 20.8MB/s 
[['CODIGO', 'DEFERIMENTO', 'NOTA_CHUM', 'NOTA_CNAT', 'NOTA_LINGUAS', 'NOTA_MAT', 'NOTA_REDACAO', 'SEXO', 'GRAU_PARENTESCO', 'ESTADO_CIVIL', 'IDADE', 'PROFISSAO', 'RENDA_MENSAL'], ['96398', 'S', '509,7', '521,3', '518,4', '445', '580', 'F', 'PAI', 'CASADO', '41', 'MOTORISTA', '2164,46'], ['96398', 'S', '509,7', '521,3', '518,4', '445', '580', 'F', 'Candidato(a)', 'SOLTEIRA', '19', 'DESEMPREGADA', ''], ['96398', 'S', '509,7', '521,3', '518,4', '445', '580', 'F', 'MÃE', 'CASADA', '41', 'DESEMPREGADA', ''], ['88410', 'S', '669,1', '585,4', '549,4', '667,6', '640', 'M', 'Pai', 'União estável', '52', 'Assalariado', '1954,71'], ['88410', 'S', '669,1', '585,4', '549,4', '667,6', '640', 'M', 'Mãe', 'União estável', '52', 'Desempregado', ''], ['88410', 'S', '669,1', '585,4', '549,4', '667,6', '640', 'M', 'Irmã(o)', 'Solteiro(a)', '24', 'Desempregado', ''], ['88410', 'S', '669,1', '585,4', '549,4', '667,6', '640', 'M', 'Candidato(a)', 'S

In [0]:
import pandas as pd
import numpy as np

df = pd.DataFrame(data[1:], columns=data[0])

print(data[0])
data_id = [did for did in df.CODIGO if did != ""]
df.RENDA_MENSAL = [val.replace("","0") if val == "" else val.replace(",",".") for val in df.RENDA_MENSAL]
df.RENDA_MENSAL = df.RENDA_MENSAL.astype(float)
# df.RENDA_MENSAL
df['CNT_FAMILIA'] = [1 for _ in df.CODIGO]
group = df.groupby(['CODIGO'])
def_= group.DEFERIMENTO
group = pd.DataFrame(group.sum())

group["DEFERIMENTO"] = ["S" if "S" in list(i[1]) else "N" for i in def_]
group['RENDA_MEDIA'] = group["RENDA_MENSAL"] / group["CNT_FAMILIA"]

print(group)
np.mean(group.RENDA_MEDIA)


print("DEFERIDOS - "+str(len(group[group.DEFERIMENTO == "S"])))
print("INDEFERIDOS - "+str(len(group[group.DEFERIMENTO == "N"])))

# print(group[group.DEFERIMENTO == "N"].RENDA_MEDIA.min())
print(group[group.DEFERIMENTO == "N"].RENDA_MEDIA.mean())
# print(group[group.DEFERIMENTO == "N"].RENDA_MEDIA.max())

# print(group[group.DEFERIMENTO == "S"].RENDA_MEDIA.min())
print(group[group.DEFERIMENTO == "S"].RENDA_MEDIA.mean())
# print(group[group.DEFERIMENTO == "S"].RENDA_MEDIA.max())

print(np.mean(group))


# import re

# rep = {",":".", "": "0"} # define desired replacements here

# # use these three lines to do the replacement
# rep = dict((re.escape(k), v) for k, v in rep.items())
# pattern = re.compile("|".join(rep.keys()))
# df.RENDA_MENSAL = [pattern.sub(lambda m: rep[re.escape(m.group(0))], val) for val in df.RENDA_MENSAL] 
# print
# df.RENDA_MENSAL = df.RENDA_MENSAL.astype(float)
# print(df.RENDA_MENSAL)



['CODIGO', 'DEFERIMENTO', 'NOTA_CHUM', 'NOTA_CNAT', 'NOTA_LINGUAS', 'NOTA_MAT', 'NOTA_REDACAO', 'SEXO', 'GRAU_PARENTESCO', 'ESTADO_CIVIL', 'IDADE', 'PROFISSAO', 'RENDA_MENSAL']
        RENDA_MENSAL  CNT_FAMILIA DEFERIMENTO  RENDA_MEDIA
CODIGO                                                    
87994        1080.00            2           S   540.000000
87995        2729.05            3           S   909.683333
87996        1414.00            3           S   471.333333
87997         700.01            2           S   350.005000
88002        2732.62            3           S   910.873333
88006        5100.00            6           N   850.000000
88009        1220.00            3           S   406.666667
88010        1908.00            4           S   477.000000
88013        2900.00            3           N   966.666667
88015         800.00            4           S   200.000000
88020        2300.00            5           S   460.000000
88021        1400.00            5           S   280.0000

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score



loo = LeaveOneOut()
loo.get_n_splits(group.index)

for c in range(2,5):
    clf = KNeighborsClassifier(c)
    for train_index, test_index in loo.split(group.index):
        
        train = group.iloc[train_index, 1:]
        print(train)
#         selection = group.index[test_index]
#         train = group.iloc(selection, ["DEFERIMENTO"])
#         
#         X_train, X_test = train[["RENDA_MEDIA", "CNT_FAMILIA", "RENDA_MENSAL"]], test[["RENDA_MEDIA", "CNT_FAMILIA", "RENDA_MENSAL"]]
#         y_train, y_test = train.DEFERIMENTO, test.DEFERIMENTO
      
#         print(X_train, y_train)
#         clf.fit(X_train, y_train)
        
#         pred = clf.predict(X_test, y_test)
#         print(clf.accuracy_score(y_train, pred))